In [18]:
# LangChain Agent with Groq and Tavily Search

# This notebook demonstrates how to create a conversational AI agent using:
# - **Groq** for fast LLM inference
# - **Tavily** for web search capabilities
# - **LangGraph** for agent orchestration

# ## Prerequisites
# Run the following command to install required packages:
# ```bash
%pip install langchain-tavily python-dotenv langchain-openai langchain-groq
# ```

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Load environment variables
from dotenv import load_dotenv
import os

# Load .env file containing API keys
load_dotenv()

# Verify API keys are loaded
groq_key = os.getenv("GROQ_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print(f"Groq API Key: {'✓ Loaded' if groq_key else '✗ Missing'}")
print(f"Tavily API Key: {'✓ Loaded' if tavily_key else '✗ Missing'}")

if not groq_key:
    print("⚠️  Add GROQ_API_KEY to your .env file")
if not tavily_key:
    print("⚠️  Add TAVILY_API_KEY to your .env file")

Groq API Key: ✓ Loaded
Tavily API Key: ✓ Loaded


In [26]:
# Import required libraries
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Initialize components
memory = MemorySaver()
model = init_chat_model("groq:llama-3.1-8b-instant", temperature=0)
search = TavilySearch(max_results=2)
tools = [search]

# Create the agent
agent_executor = create_react_agent(model, tools, checkpointer=memory)

print("✅ Agent created successfully!")

✅ Agent created successfully!


C:\Users\ASUS\AppData\Local\Temp\ipykernel_18788\2083565201.py:14: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(model, tools, checkpointer=memory)


In [27]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    final_message=step["messages"][-1]

final_message.pretty_print()

================================== Ai Message ==================================

Nice to meet you, Bob. What can I help you with today?


In [28]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (33as33k9q)
 Call ID: 33as33k9q
  Args:
    query: San Francisco weather today
    search_depth: basic
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "San Francisco weather today", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, California", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1760965753, 'localtime': '2025-10-20 06:09'}, 'current': {'last_updated_epoch': 1760965200, 'last_updated': '2025-10-20 06:00', 'temp_c': 12.2, 'temp_f': 